
# 🧠 Caso de Uso ML en Cadena de Suministro: Pronóstico de Demanda (M5, Walmart)
**Notebook base listo para correr** — Incluye: descarga de datos (opcional vía Kaggle), EDA mínima, *feature engineering*, validación temporal con ventana rodante, entrenamiento con **LightGBM**, métricas (**MAE, RMSE, WMAPE**), e inferencia de un horizonte de 28 días.

> **Requisitos (elige tu ruta):**
> 1) **Colab + Kaggle API** (recomendado): sube tu `kaggle.json` y corre la celda de descarga.  
> 2) **Local (descarga manual)**: descarga el dataset **M5 Forecasting - Accuracy** desde Kaggle y descomprime los CSV en una carpeta local.  
>
> **Archivos esperados (`calendar.csv`, `sales_train_validation.csv`, `sell_prices.csv`)** del dataset **M5 Forecasting - Accuracy**.
>
> Para acelerar en equipos modestos, puedes limitar el número de SKU/tienda con `N_ITEMS_SAMPLE`.


In [ ]:

# @title Instalar dependencias (ejecuta si usas Colab o un entorno limpio)
# Si estás en local y ya tienes las librerías, puedes omitir.
# Nota: En Colab se recomienda reiniciar el runtime tras instalar.
try:
    import pandas as pd
    import numpy as np
    import lightgbm as lgb
    import matplotlib.pyplot as plt
except Exception as e:
    !pip -q install pandas numpy lightgbm scikit-learn matplotlib pyarrow python-dateutil


In [ ]:

# @title Imports y configuración global
import os, gc, warnings, math, json, pathlib
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 120)

SEED = 42
np.random.seed(SEED)

print("Versiones -> pandas:", pd.__version__)



## ⚙️ Configuración
- `DATA_DIR`: carpeta donde están los CSV del M5 (`calendar.csv`, `sales_train_validation.csv`, `sell_prices.csv`).
- `USE_KAGGLE_DOWNLOAD`: si `True`, descargará desde Kaggle (requiere `kaggle.json`).
- `N_ITEMS_SAMPLE`: número de SKU a muestrear para pruebas rápidas (None = usar todos).  
- `STATE_FILTER` / `STORE_FILTER`: filtra por estado o tienda (útil para acelerar).  
- `HORIZON`: horizonte de predicción (M5 = 28 días).


In [ ]:

# @title Parámetros de ejecución
DATA_DIR = "./m5_data"        # Cambia a tu ruta local si usas descarga manual
USE_KAGGLE_DOWNLOAD = False   # Pon True si quieres descargar con Kaggle API
KAGGLE_DATASET = "m5-forecasting-accuracy"  # M5 (Accuracy)

# Submuestreo para acelerar en pruebas (None usa todo)
N_ITEMS_SAMPLE = 500          # e.g., 500 SKU; pon None para todos
STATE_FILTER   = None         # e.g., "CA"
STORE_FILTER   = None         # e.g., "CA_1"

HORIZON = 28                  # Horizonte oficial M5
N_FOLDS = 2                   # Nº de cortes de validación rodante
MAX_ESTIMATORS = 1500         # Árboles LGBM
EARLY_STOP = 100


In [ ]:

# @title (Opcional) Descarga Kaggle - Requiere subir tu kaggle.json
# Ejecuta SOLO si USE_KAGGLE_DOWNLOAD = True
if USE_KAGGLE_DOWNLOAD:
    os.makedirs(DATA_DIR, exist_ok=True)
    # Instalar kaggle si es necesario
    try:
        import kaggle
    except:
        !pip -q install kaggle

    # Configurar credenciales (sube tu kaggle.json a la carpeta actual)
    # En Colab: arrastra kaggle.json o ejecuta desde drive.
    KAGGLE_JSON = "kaggle.json"
    if not os.path.exists(KAGGLE_JSON):
        print("⚠️ Sube tu kaggle.json (desde tu cuenta de Kaggle) al directorio actual y reintenta.")
    else:
        os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
        import shutil, stat
        shutil.copy(KAGGLE_JSON, os.path.expanduser("~/.kaggle/kaggle.json"))
        os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), stat.S_IRUSR | stat.S_IWUSR)
        # Descargar y descomprimir
        !kaggle competitions download -c {KAGGLE_DATASET} -p {DATA_DIR}
        # Archivos a extraer: calendar.csv, sales_train_validation.csv, sell_prices.csv
        import zipfile, glob
        for z in glob.glob(os.path.join(DATA_DIR, "*.zip")):
            print("Descomprimiendo:", z)
            with zipfile.ZipFile(z, "r") as zip_ref:
                zip_ref.extractall(DATA_DIR)
        print("✅ Descarga y extracción completadas en:", DATA_DIR)



## 📥 Carga de datos (M5)
Se utilizan los 3 CSV principales:
- `calendar.csv` (calendario, eventos, SNAP), 
- `sales_train_validation.csv` (ventas por SKU-tienda en formato ancho `d_1..d_n`),
- `sell_prices.csv` (precios por semana/SKU/tienda).


In [ ]:

# @title Cargar CSV del M5
def memory_usage(df):
    return f"{df.memory_usage(deep=True).sum()/1024**2:,.2f} MB"

def load_m5_data(data_dir):
    cal_types = {
        "event_name_1":"category", "event_type_1":"category",
        "event_name_2":"category", "event_type_2":"category",
        "weekday":"category", "wm_yr_wk":"int32",
        "wday":"int8", "month":"int8", "year":"int16", "snap_CA":"int8", "snap_TX":"int8", "snap_WI":"int8"
    }
    calendar = pd.read_csv(os.path.join(data_dir, "calendar.csv"), dtype=cal_types, parse_dates=["date"])

    price_types = {"store_id":"category", "item_id":"category", "wm_yr_wk":"int32", "sell_price":"float32"}
    sell_prices = pd.read_csv(os.path.join(data_dir, "sell_prices.csv"), dtype=price_types)

    # Ventas: formato ancho (d_1..d_n). Reducimos dtypes categóricos
    sale_types = {
        "item_id":"category","dept_id":"category","cat_id":"category","store_id":"category","state_id":"category"
    }
    sales = pd.read_csv(os.path.join(data_dir, "sales_train_validation.csv"), dtype=sale_types)
    return calendar, sell_prices, sales

assert os.path.exists(DATA_DIR), f"No se encontró DATA_DIR={DATA_DIR}. Ajústalo o usa Kaggle download."
calendar, sell_prices, sales = load_m5_data(DATA_DIR)

print("calendar:", calendar.shape, "|", memory_usage(calendar))
print("sell_prices:", sell_prices.shape, "|", memory_usage(sell_prices))
print("sales:", sales.shape, "|", memory_usage(sales))

display(calendar.head(2))
display(sell_prices.head(2))
display(sales.head(2))



## 🔎 Filtro y submuestreo (opcional para acelerar)
Puedes filtrar por `STATE_FILTER`/`STORE_FILTER`, o muestrear `N_ITEMS_SAMPLE` SKU.


In [ ]:

# @title Aplicar filtros / submuestreo
def filter_and_sample_sales(df, state=None, store=None, n_items_sample=None, seed=42):
    if state is not None:
        df = df[df["state_id"]==state]
    if store is not None:
        df = df[df["store_id"]==store]
    if n_items_sample is not None:
        # muestrear por item_id dentro del filtro
        items = df["item_id"].unique()
        rng = np.random.default_rng(seed)
        take = n_items_sample if n_items_sample < len(items) else len(items)
        sample_items = rng.choice(items, size=take, replace=False)
        df = df[df["item_id"].isin(sample_items)]
    return df

sales_filtered = filter_and_sample_sales(
    sales.copy(), state=STATE_FILTER, store=STORE_FILTER, n_items_sample=N_ITEMS_SAMPLE, seed=SEED
)
print("Sales filtrado:", sales_filtered.shape, "|", memory_usage(sales_filtered))
display(sales_filtered.head(2))



## 🔁 Transformación wide → long y *join* con calendario y precios
- Pasamos de columnas `d_1..d_n` a filas por día.  
- Unimos con `calendar` por `d`/`date`.  
- Unimos con `sell_prices` por `store_id, item_id, wm_yr_wk`.


In [ ]:

# @title Wide → Long y uniones
# Identificamos columnas d_*
d_cols = [c for c in sales_filtered.columns if c.startswith("d_")]
id_cols = [c for c in sales_filtered.columns if c not in d_cols]

long_df = sales_filtered.melt(id_vars=id_cols, value_vars=d_cols, var_name="d", value_name="sales")
long_df["sales"] = long_df["sales"].astype("float32")

# Unir con calendario (obtener fecha y wm_yr_wk)
cal_small = calendar[["d","date","wm_yr_wk","wday","month","year","snap_CA","snap_TX","snap_WI","event_name_1","event_type_1"]].copy()
long_df = long_df.merge(cal_small, on="d", how="left")

# Unir con precios
long_df = long_df.merge(sell_prices, on=["store_id","item_id","wm_yr_wk"], how="left")

# Rellenar precios faltantes con último valor conocido por item_id/store
long_df["sell_price"] = long_df.groupby(["store_id","item_id"])["sell_price"].transform(lambda s: s.ffill().bfill())
long_df["sell_price"] = long_df["sell_price"].astype("float32")

print("long_df:", long_df.shape, "|", memory_usage(long_df))
display(long_df.head(3))



## 🧩 *Feature engineering* clave
- **Lags**: `lag_7`, `lag_28`  
- **Rolling windows**: `rmean_7`, `rmean_28`, `rstd_7`, `rstd_28`  
- **Precio**: `sell_price`, `price_change_rate`, `price_rel_rolling_28`  
- **Calendario**: `wday`, `month`, `year`, `snap_state`, `event_type_1` (codificado)


In [ ]:

# @title Crear features (lags, rolling, precio, calendario)
def add_calendar_features(df):
    df["wday"] = df["wday"].astype("int8")
    df["month"] = df["month"].astype("int8")
    df["year"] = df["year"].astype("int16")
    # SNAP según estado de la tienda
    df["snap"] = 0
    df.loc[df["state_id"]=="CA","snap"] = df.loc[df["state_id"]=="CA","snap_CA"]
    df.loc[df["state_id"]=="TX","snap"] = df.loc[df["state_id"]=="TX","snap_TX"]
    df.loc[df["state_id"]=="WI","snap"] = df.loc[df["state_id"]=="WI","snap_WI"]
    df["snap"] = df["snap"].fillna(0).astype("int8")
    # Eventos (codificación simple)
    df["event_type_1"] = df["event_type_1"].astype("category").cat.codes.astype("int16")
    return df

def add_lag_roll_features(df, group_cols=["item_id","store_id"], sales_col="sales"):
    df = df.sort_values(group_cols + ["date"])
    # Lags
    for L in [7, 28]:
        df[f"lag_{L}"] = df.groupby(group_cols)[sales_col].shift(L)
    # Rolling means/std
    df["rmean_7"]  = df.groupby(group_cols)["sales"].transform(lambda s: s.shift(1).rolling(7).mean())
    df["rmean_28"] = df.groupby(group_cols)["sales"].transform(lambda s: s.shift(1).rolling(28).mean())
    df["rstd_7"]   = df.groupby(group_cols)["sales"].transform(lambda s: s.shift(1).rolling(7).std())
    df["rstd_28"]  = df.groupby(group_cols)["sales"].transform(lambda s: s.shift(1).rolling(28).std())
    # Precio
    df["price_change_rate"] = df.groupby(group_cols)["sell_price"].pct_change().replace([np.inf,-np.inf],0).fillna(0).astype("float32")
    df["price_roll28"] = df.groupby(group_cols)["sell_price"].transform(lambda s: s.rolling(28, min_periods=1).mean())
    df["price_rel_rolling_28"] = (df["sell_price"] / df["price_roll28"]).astype("float32")
    return df

long_df = add_calendar_features(long_df)
long_df = add_lag_roll_features(long_df)

# Eliminar filas con NaN generados por lags/rolling (calentamiento)
min_date = long_df["date"].min()
cutoff = long_df["date"].min() + pd.Timedelta(days=60)
long_df = long_df[long_df["date"]>=cutoff].copy()

print("long_df (con features):", long_df.shape, "|", memory_usage(long_df))
display(long_df.head(3))



## ⏳ Validación temporal (ventana rodante)
Creamos cortes de entrenamiento/validación manteniendo integridad temporal. Cada *fold* valida **HORIZON** días al final de la ventana.


In [ ]:

# @title Crear cortes de backtesting
def make_time_folds(df, date_col="date", horizon=28, n_folds=2):
    last_date = df[date_col].max()
    folds = []
    for i in range(n_folds):
        valid_end = last_date - pd.Timedelta(days=horizon*i)
        valid_start = valid_end - pd.Timedelta(days=horizon-1)
        train_end = valid_start - pd.Timedelta(days=1)
        # usamos todo lo anterior para entrenar
        folds.append({
            "train_end": train_end,
            "valid_start": valid_start,
            "valid_end": valid_end
        })
    folds = folds[::-1]  # del más antiguo al más reciente
    return folds

folds = make_time_folds(long_df, "date", horizon=HORIZON, n_folds=N_FOLDS)
folds



## 🌲 Modelo: LightGBM (global por SKU‑tienda)
Entrenamos un único modelo con *features* comunes para todas las series (aprendizaje global).


In [ ]:

# @title Entrenar y evaluar con backtesting
import lightgbm as lgb

FEATURES = [
    "wday","month","year","snap","event_type_1",
    "sell_price","price_change_rate","price_rel_rolling_28",
    "lag_7","lag_28","rmean_7","rmean_28","rstd_7","rstd_28"
]
TARGET = "sales"

def wmape(y_true, y_pred):
    denom = np.abs(y_true).sum()
    return np.inf if denom == 0 else (np.abs(y_true - y_pred).sum() / denom)

metrics_cv = []
models = []
feature_importances = []

for k, f in enumerate(folds, 1):
    train_mask = (long_df["date"] <= f["train_end"])
    valid_mask = (long_df["date"] >= f["valid_start"]) & (long_df["date"] <= f["valid_end"])

    train_df = long_df.loc[train_mask, FEATURES + [TARGET]].dropna()
    valid_df = long_df.loc[valid_mask, FEATURES + [TARGET]].dropna()

    X_tr, y_tr = train_df[FEATURES], train_df[TARGET]
    X_va, y_va = valid_df[FEATURES], valid_df[TARGET]

    model = lgb.LGBMRegressor(
        n_estimators=MAX_ESTIMATORS,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=SEED,
        n_jobs=-1
    )
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="l2",
        verbose=False
    )

    y_hat = model.predict(X_va)
    mae  = mean_absolute_error(y_va, y_hat)
    rmse = mean_squared_error(y_va, y_hat, squared=False)
    wmp  = wmape(y_va.values, y_hat)
    metrics_cv.append({"fold": k, "MAE": mae, "RMSE": rmse, "WMAPE": wmp})

    # Importancias
    fi = pd.Series(model.feature_importances_, index=FEATURES, name=f"fold_{k}")
    feature_importances.append(fi)

    models.append(model)
    print(f"[Fold {k}] MAE={mae:,.3f} | RMSE={rmse:,.3f} | WMAPE={wmp:,.3%}  (Train end: {f['train_end'].date()}, Valid: {f['valid_start'].date()}→{f['valid_end'].date()})")

metrics_cv = pd.DataFrame(metrics_cv)
display(metrics_cv)
print("Promedios CV:", metrics_cv.mean().to_dict())

# Importancia promedio
fi_df = pd.concat(feature_importances, axis=1).mean(axis=1).sort_values(ascending=False).to_frame("importance")
display(fi_df)



## 📈 Visualizar predicción vs. real (ejemplo por un SKU‑tienda)
Seleccionamos un par `item_id`/`store_id` con datos en la última validación y graficamos.


In [ ]:

# @title Gráfica de ejemplo
last_fold = folds[-1]
valid_mask = (long_df["date"] >= last_fold["valid_start"]) & (long_df["date"] <= last_fold["valid_end"])
valid_rows = long_df.loc[valid_mask].dropna(subset=FEATURES + [TARGET]).copy()

# Tomar un par SKU-tienda con variación
grp_cols = ["item_id","store_id"]
pair = (valid_rows.groupby(grp_cols)["sales"].sum().sort_values(ascending=False).index[0])
pair_df = valid_rows[(valid_rows["item_id"]==pair[0]) & (valid_rows["store_id"]==pair[1])].copy()

X_pair = pair_df[FEATURES]
y_pair = pair_df[TARGET].values

# Usamos el último modelo entrenado
y_hat_pair = models[-1].predict(X_pair)

plot_df = pair_df[["date"]].copy()
plot_df["y_true"] = y_pair
plot_df["y_hat"] = y_hat_pair

plt.figure(figsize=(10,4))
plt.plot(plot_df["date"], plot_df["y_true"], label="Real")
plt.plot(plot_df["date"], plot_df["y_hat"], label="Pronóstico")
plt.title(f"SKU: {pair[0]} | Tienda: {pair[1]}")
plt.legend()
plt.tight_layout()
plt.show()



## 🚀 Entrenamiento final y pronóstico futuro (28 días)
Entrenamos con todo el histórico disponible y proyectamos **HORIZON** días hacia adelante, actualizando *lags* de forma iterativa.  
> Nota: Para características futuras (eventos/SNAP) usamos el `calendar.csv` (ya incluye fechas posteriores a las últimas ventas). Para precios, mantenemos el último valor conocido.


In [ ]:

# @title Pronóstico futuro
final_model = lgb.LGBMRegressor(
    n_estimators=MAX_ESTIMATORS,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=SEED,
    n_jobs=-1
)
train_full = long_df.dropna(subset=FEATURES + [TARGET])
X_full, y_full = train_full[FEATURES], train_full[TARGET]
final_model.fit(X_full, y_full)

# Última fecha observada
last_date = long_df["date"].max()
future_dates = pd.date_range(last_date + pd.Timedelta(days=1), periods=HORIZON, freq="D")

# Base para futuro: replicamos claves y unimos con calendario
keys = long_df[["item_id","store_id","state_id","dept_id","cat_id"]].drop_duplicates()
cal_fut = calendar[calendar["date"].isin(future_dates)][["date","wm_yr_wk","wday","month","year","snap_CA","snap_TX","snap_WI","event_type_1"]]
future_df = (
    keys.assign(key=1)
    .merge(cal_fut.assign(key=1), on="key", how="left")
    .drop(columns=["key"])
    .merge(sell_prices, on=["store_id","item_id","wm_yr_wk"], how="left")
)

# Rellenar precios con último valor histórico por SKU-tienda
last_prices = long_df.groupby(["store_id","item_id"])["sell_price"].last().rename("last_price").reset_index()
future_df = future_df.merge(last_prices, on=["store_id","item_id"], how="left")
future_df["sell_price"] = future_df["sell_price"].fillna(future_df["last_price"]).astype("float32")
future_df.drop(columns=["last_price"], inplace=True)

# Añadir calendario, y preparar columnas necesarias
future_df = add_calendar_features(future_df)

# Construiremos lags/rolling de forma iterativa usando una copia con histórico + futuro
hist_cols = ["date","item_id","store_id","sales","sell_price","wday","month","year","snap","event_type_1"]
needed_cols = hist_cols + ["lag_7","lag_28","rmean_7","rmean_28","rstd_7","rstd_28","price_change_rate","price_roll28","price_rel_rolling_28"]

# Tomamos histórico mínimo para poder calcular lags/rolling
hist = long_df[["date","item_id","store_id","sales","sell_price","wday","month","year","snap","event_type_1"]].copy()
comb = pd.concat([hist, future_df[["date","item_id","store_id","sell_price","wday","month","year","snap","event_type_1"]].assign(sales=np.nan)], ignore_index=True)
comb = comb.sort_values(["item_id","store_id","date"]).reset_index(drop=True)

# Funciones auxiliares para lags/rolling en loop
def update_lags_roll(df):
    df = df.sort_values(["item_id","store_id","date"])
    for L in [7,28]:
        df[f"lag_{L}"] = df.groupby(["item_id","store_id"])["sales"].shift(L)
    df["rmean_7"]  = df.groupby(["item_id","store_id"])["sales"].transform(lambda s: s.shift(1).rolling(7).mean())
    df["rmean_28"] = df.groupby(["item_id","store_id"])["sales"].transform(lambda s: s.shift(1).rolling(28).mean())
    df["rstd_7"]   = df.groupby(["item_id","store_id"])["sales"].transform(lambda s: s.shift(1).rolling(7).std())
    df["rstd_28"]  = df.groupby(["item_id","store_id"])["sales"].transform(lambda s: s.shift(1).rolling(28).std())
    df["price_change_rate"] = df.groupby(["item_id","store_id"])["sell_price"].pct_change().replace([np.inf,-np.inf],0).fillna(0).astype("float32")
    df["price_roll28"] = df.groupby(["item_id","store_id"])["sell_price"].transform(lambda s: s.rolling(28, min_periods=1).mean())
    df["price_rel_rolling_28"] = (df["sell_price"] / df["price_roll28"]).astype("float32")
    return df

comb = update_lags_roll(comb)

# Pronóstico iterativo día a día
preds = []
for d in future_dates:
    mask = comb["date"]==d
    X = comb.loc[mask, FEATURES].copy()
    yhat = final_model.predict(X)
    comb.loc[mask, "sales"] = yhat
    preds.append(pd.DataFrame({"date": d, "item_id": comb.loc[mask,"item_id"].values,
                               "store_id": comb.loc[mask,"store_id"].values, "yhat": yhat}))
    # Actualizamos lags/rolling para el siguiente día
    comb = update_lags_roll(comb)

forecast_df = pd.concat(preds, ignore_index=True)
print("Pronóstico futuro:", forecast_df.shape)
display(forecast_df.head())

# Guardar resultados
os.makedirs("outputs", exist_ok=True)
forecast_path = "outputs/forecast_m5_lgbm.csv"
metrics_path  = "outputs/cv_metrics.csv"
fi_path       = "outputs/feature_importance.csv"

forecast_df.to_csv(forecast_path, index=False)
metrics_cv.to_csv(metrics_path, index=False)
fi_df.to_csv(fi_path)

print("Archivos guardados:")
print("-", forecast_path)
print("-", metrics_path)
print("-", fi_path)



## 📊 Visual rápido del pronóstico agregado (ejemplo)
Graficamos demanda total pronosticada agregada por fecha.


In [ ]:

# @title Gráfico del total pronosticado
agg_forecast = forecast_df.groupby("date")["yhat"].sum().reset_index()
plt.figure(figsize=(10,4))
plt.plot(agg_forecast["date"], agg_forecast["yhat"])
plt.title("Pronóstico total (suma de yhat)")
plt.tight_layout()
plt.show()



---

### ✅ Siguientes pasos sugeridos
- Ajustar hiperparámetros (grid/bayes) y probar **quantile regression** (P50/P90) con LightGBM para intervalos.  
- Incorporar características adicionales: festividades específicas, *fourier terms*, elasticidades por precio, *hierarchical reconciliation*.  
- Conectar este pronóstico a una **simulación de inventario** para calcular **stock de seguridad**, **fill-rate** y **costo total**.

**Autor:** Generado por ChatGPT · Semilla `SEED=42`  
